# tennis ball tracker

## packages

In [1]:
import cv2
import numpy as np
from modules import proc
from modules import tracking

## initialization

In [2]:
video_path = './videos/videoplayback.mp4'
bRecord = False
que = proc.ImgQueue(5)

kernel_size = 30
kernel_dilation = cv2.getStructuringElement(cv2.MORPH_RECT,(kernel_size,kernel_size))
kernel_open = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))

width = 640.

if bRecord == True:
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('{}.avi'.format(video_path.split('/')[-1].split('.')[0]),fourcc, 20.0, (640,360))

## run

In [3]:
cap = cv2.VideoCapture(video_path)
tg = tracking.track_manager(50, 5, 5)

while(cap.isOpened()):
    ret, img = cap.read()
    try:
        resize_scale = width / float(img.shape[1])
        img = cv2.resize(img, None, fx=resize_scale, fy=resize_scale)
        img_debug = np.copy(img)

        # put img into queue
        img_avg = que.calculate_average().astype(np.uint8)    
        que.enqueue(img)    

        # background subtraction
        img_diff = np.abs(img.astype(np.float)-img_avg.astype(np.float)).astype(np.uint8)
        img_diff_gray = cv2.cvtColor(img_diff, cv2.COLOR_BGR2GRAY)
        ret, img_diff_gray = cv2.threshold(img_diff_gray, 50, 255, cv2.THRESH_BINARY) 
        img_diff_gray = cv2.morphologyEx(img_diff_gray, cv2.MORPH_CLOSE, kernel_open)

        # dilation to remove players
        img_dilation = cv2.morphologyEx(img_diff_gray, cv2.MORPH_DILATE, kernel_dilation)    
        clusters_dilation, _ = proc.clustering(img_dilation, None, None)

        # eliminate players
        for cluster in clusters_dilation:
            if cluster[4] < 1500:
                img_dilation[cluster[1]:cluster[1]+cluster[3], cluster[0]:cluster[0]+cluster[2]] = 0
        img_diff_gray = img_diff_gray - img_dilation    
        ret, img_diff_gray = cv2.threshold(img_diff_gray, 127, 255, cv2.THRESH_BINARY) 

        clusters_diff, clusters_diff_center = proc.clustering(img_diff_gray, (2, 30), (0.5, 2.0))

        # tracking
        tg.manage(clusters_diff_center)

        # display    
        tracks = tg.get_valid_tracks()
        for i in range(len(tracks)):
            t = tracks[i]
            pos = (int(t.pos[0]), int(t.pos[1]))
            cv2.circle(img_debug, pos, 2, (0,0,255), -1)

            if len(t.traj) > 1:
                for i in range(len(t.traj)-1):
                    pt1 = (int(t.traj[i][0]), int(t.traj[i][1]))
                    pt2 = (int(t.traj[i+1][0]), int(t.traj[i+1][1]))       
                    cv2.circle(img_debug, pt1, 3, (0,0,255), -1)
                    cv2.circle(img_debug, pt2, 3, (0,255,0), -1)
                    cv2.line(img_debug, pt1, pt2, (0,200,128), 2)

        cv2.namedWindow('debug')
        cv2.imshow('debug',img_debug)

        # record
        if bRecord == True:
            out.write(img_debug)

        # terminate
        k = cv2.waitKey(1) 
        if k == 27:
            cv2.destroyAllWindows()
            cv2.waitKey(1)
            break
            
    except AttributeError:
        break

cap.release()

if bRecord == True:
    out.release()

cv2.destroyAllWindows()
cv2.waitKey(1) 

/anaconda2/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


-1